In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [25]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import seaborn as sns
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
from scipy.stats.stats import ttest_ind
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [3]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "useducation"

<IPython.core.display.Javascript object>

In [5]:
engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

df = pd.read_sql_query("select * from useducation", con=engine)
engine.dispose()

<IPython.core.display.Javascript object>

In [13]:
pd.set_option("display.max_columns", None)
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


<IPython.core.display.Javascript object>

In [12]:
# df.info()

<IPython.core.display.Javascript object>

In [11]:
df2 = df.copy()
fill_list = [
    "ENROLL",
    "TOTAL_REVENUE",
    "FEDERAL_REVENUE",
    "STATE_REVENUE",
    "LOCAL_REVENUE",
    "TOTAL_EXPENDITURE",
    "INSTRUCTION_EXPENDITURE",
    "SUPPORT_SERVICES_EXPENDITURE",
    "OTHER_EXPENDITURE",
    "CAPITAL_OUTLAY_EXPENDITURE",
    "GRADES_PK_G",
    "GRADES_KG_G",
    "GRADES_4_G",
    "GRADES_8_G",
    "GRADES_12_G",
    "GRADES_1_8_G",
    "GRADES_9_12_G",
    "GRADES_ALL_G",
]
for col in fill_list:
    df2[col] = df2.groupby("STATE")[col].apply(
        lambda x: x.interpolate(limit_direction="both")
    )
df2 = df2.dropna()
# df2.info()

<IPython.core.display.Javascript object>

## 1 ##

Compute the average score for each row for all students weighted properly by grade. In other words, each row will have its own average. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores.


In [32]:
df2["w_average"] = (df2["GRADES_4_G"]*((df2["AVG_MATH_4_SCORE"] + df2["AVG_READING_4_SCORE"])*0.5) + df2["GRADES_8_G"]
                                 * ((df2["AVG_MATH_8_SCORE"] + df2["AVG_READING_8_SCORE"])*0.5))/(df2["GRADES_4_G"] + df2["GRADES_8_G"])


<IPython.core.display.Javascript object>

## 2 ## 

What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?


In [33]:
exp_df=df2[["w_average", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

exp_df.corr()[["w_average"]].sort_values(by=["w_average"], ascending=False)


,w_average
w_average,1.000000
INSTRUCTION_EXPENDITURE,0.102186
SUPPORT_SERVICES_EXPENDITURE,0.094281
TOTAL_EXPENDITURE,0.089278
OTHER_EXPENDITURE,0.027234
CAPITAL_OUTLAY_EXPENDITURE,0.009252


<IPython.core.display.Javascript object>

In [34]:
exp_df

,w_average,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
2,235.639070,3401580.0,1435908.0,1007732.0,164083.0,609114.0
3,235.441283,1743022.0,964323.0,483488.0,68617.0,145212.0
5,241.797898,3264826.0,1642466.0,1035970.0,92163.0,364760.0
6,245.559478,3721338.0,2148041.0,1142600.0,143312.0,48542.0
7,236.784089,638784.0,372722.0,194915.0,26548.0,30595.0
...,...,...,...,...,...,...
1487,259.594501,16113212.0,8755896.0,5075509.0,627473.0,1086722.0
1488,255.009879,13630138.0,6508964.0,4510672.0,546926.0,1601069.0
1489,249.950079,3466981.0,1819903.0,1161944.0,233836.0,232738.0
1490,251.726455,11553677.0,5723474.0,3691809.0,513402.0,894823.0


<IPython.core.display.Javascript object>

Instruction expenditure has the highest corr

## 3 ##

Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?


In [35]:
X = exp_df.drop(columns=["w_average"])
y = exp_df["w_average"]

<IPython.core.display.Javascript object>

In [50]:
X=StandardScaler().fit_transform(X)
pca=PCA(n_components=1)

exp_df['pca_col']=pca.fit_transform(X)
pca.explained_variance_ratio_


array([0.95203581])

<IPython.core.display.Javascript object>

## 4 ##
What is the correlation between the overall score variable and the 1st principal component?


In [51]:
from scipy.stats.stats import pearsonr

exp_df["pca_col"].corr(y)

0.06658624125519856

<IPython.core.display.Javascript object>

The correlation is rather low

## 5 ##

If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?









Well the 1st PC epxlained a lot of teh variance of the target variable, but its constituents are highly correlated with each other so it may be unstable.

EXTRA Question analysis:


In [58]:
df.dropna().GRADES_8_G.sum() - df.dropna().GRADES_4_G.sum()

277785.0

<IPython.core.display.Javascript object>